In [1]:
import stix_notebook

## Simplest Example

The simplest thing you can generate is a single STIX object. Do this by specifying the object type name, capitalized and with spaces replaced by underscores. All statements must be terminated with a period.

In [2]:
%%stix
Threat_Actor.

In addition to STIX Domain Objects (SDOs), we can also generate STIX Cyber Observables (SCOs).

In [3]:
%%stix
Domain_Name.

## Relationships

You can create relationships by giving the capitalized source object type, followed by the lowercase relationship type, followed by the target object type. If the relationship type has a space in it, replace it with a hyphen ("-").

In [4]:
%%stix
Malware indicates Indicator.

## Counts

You can create multiple objects of the same type using a numeral before the object type name.

In [5]:
%%stix
2 Domain_Name.

To create multiple object types at once, you can use multiple statements, each terminated with a period.

In [6]:
%%stix
Campaign. 2 Indicator.

You can also generate multiple objects by surrounding them in parentheses:

In [7]:
%%stix
(Campaign 2 Indicator).

## Relationships With Counts

You can create multiple relationships at once as well.

In [8]:
%%stix
Attack_Pattern 2 uses Malware.

In [9]:
%%stix
Campaign targets (Identity Vulnerability).

## Variables

You can give names to specific instances of objects by using a colon after the name, which must be lowercase. This lets you refer to them in other relationships, so the same object can be referenced multiple times.

In [10]:
%%stix
sam: Threat_Actor.
sam uses Attack_Pattern.
sam attributed-to Identity.
(sam 2 Threat_Actor) uses Malware.

Variables can refer to groups of objects as well. Note that the count goes before the variable name.

In [11]:
%%stix
2 group: Indicator.
group indicates Malware.

## Specific Property Values

You can set properties to specific values by inserting a block of properties after the object type or variable. This block is surrounded by curly braces, and contains a property name followed by a colon and then the value for that property. Multiple properties are separated by commas.

For example, to specify the name of an Identity instead of randomly generating a name:

In [12]:
%%stix
Identity {name: "Agent Smith"}.

Here is an example specifying multiple properties:

In [13]:
%%stix
Identity {name: "Agent Smith", sectors: ["technology"]}.

Note that the property block must go before the colon when declared on a variable.

In [14]:
%%stix
example {name: "example.com", pattern: "[url:value = 'http://example.com/']"}: Indicator.

## Special Relationships

Some relationships constitute special cases that are handled slightly differently than other relationships.

### object_refs
Objects that have an `object_refs` property, such as Report or Opinion, can populate this property using the "on" keyword. 

In [15]:
%%stix
Report on 3 Indicator.

In [16]:
%%stix
Opinion on (1 Campaign 2 Indicator).

### Sightings

Similarly, create sightings of objects using Sighting as the source object type and "of" as the relationship keyword.

In [17]:
%%stix
Sighting of Indicator.

### Embedded Relationships

Embedded relationships can be set like any other property, using the curly brace syntax with multiple embedded relationships separated by commas.

In [18]:
%%stix
Sighting {created_by_ref: Identity, observed_data_refs: Observed_Data} of Malware.

## Chaining

Objects can also be used as both a source and a target of two different relationships, effectively "chaining" them together.

In [19]:
%%stix
Campaign uses Malware targets Identity.

As a point of comparison, here is how the same example could be written using variables. Note that using chaining keeps the input shorter and simpler.

In [20]:
%%stix
mlwr: Malware.
Campaign uses mlwr.
mlwr targets Identity.

## Configuration

The generator has several options that can be configured by modifying the `config.ini` file in this directory.

#### string_length_min
Randomly generated strings can be no shorter than this many characters.

#### string_length_max
Randomly generated strings can be no longer than this many characters.

#### string_chars
Randomly generated strings can only include these characters.

#### number_min
Randomly generated numbers must be higher than this number.

#### is_number_min_exclusive
Whether randomly generated numbers can be equal to the `number_min` setting.

#### number_max
Randomly generated numbers must be lower than this number.

#### is_number_max_exclusive
Whether randomly generated numbers can be equal to the `number_max` setting.

#### array_length_min
Randomly generated array must contain at least this many items.

#### array_length_max
Randomly generated array must contain at most this many items.

#### optional_property_probability
Probability, between 0 and 1, of generating optional properties in the objects.

#### minimize_ref_properties
If True, the generator will only include `*_ref/s` properties if they are required properties, in an attempt to keep the generated graphs from ballooning in size.


## Custom Objects and Properties

The generator is able to generate custom objects or custom properties on existing objects, but requires editing the `custom_registry.json` file located in this directory. This file is a mapping of object type names to a variety of options and a listing of the object's properties. By default we've added a custom property to the `User_Account` type and a custom object type called `Foobar`. You can view the specifications of all the predefined STIX 2.1 object types in `stix2generator/stix21_registry.json`. Here is what the default custom registry looks like:

```json
{
    "Foobar": {
        "type": "object",
        "import": "common-properties",
        "required": ["id", "some-property", "type"],
        "properties": {
            "type": "x-foobar",
            "id": {
                "type": "string",
                "semantics": "stix-id",
                "stix-type": "x-foobar"
            },
            "some-property": {
                "type": "string",
                "semantics": "word"
            },
            "another-property": {
                "type": "string",
                "semantics": "sentence"
            }
        }
    },
    "User_Account": {
        "type": "object",
        "required": ["id", "type", "all-props-group", "x_foo"],
        "properties": {
            "x_foo": {
                "type": "string"
            }
        }
    }
}
```

Once we add a custom property to an existing object in the builder's registry, objects of that type will include that property.

In [23]:
%%stix
User_Account.

Once we've added a new object type to the builder's registry, we can generate objects of our custom type.

In [24]:
%%stix
Foobar.